In [1]:
import yt_dlp as youtube_dl
import pandas as pd
import os
import numpy as np

csv_file = "red-team-training.csv"

df = pd.read_csv(csv_file)

# df.head()

### getting the video URL from thinkific DB

In [32]:
def get_video_url(url_code):
    video_url = f"https://platform.thinkific.com/videoproxy/v1/play/{url_code}"
    return video_url

### Function to get the public URL for the video to make it downloadable

In [33]:
def get_download_url(video_url):
    print("video_url--------", video_url)
    ydl_opts = {
        "format": "best",
        "noplaylist": True,
    }

    original_video_url = None

    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        info_dict = ydl.extract_info(video_url, download=False)
        # print("\n\ninfo_dict--------\n", info_dict)
        original_video_dict = info_dict["entries"][0]["formats"]
        # print("Original video URL:", original_video_dict)
        for entry in info_dict.get("entries", []):
            for format in entry.get("formats",[]):  
                if format["format_id"] == "original":
                        original_video_url = format["url"]
                        break

        if original_video_url:
            original_video_url = original_video_url.replace(".bin", " ")
            return original_video_url
        else:
            print("Format 'Original Url' not found.")
            return None

### Append the download URL to csv for the thinkific video

In [ ]:
if 'video_url' not in df.columns:
    df['video_url'] = None 

if 'download_url' not in df.columns:
    df['download_url'] = None 

for index,row in df.iterrows():
    if pd.isna(row['download_url']) or row['download_url'] == "":
        print("Fetching video link for ",row['chapter_name'])
        video_url = get_video_url(row['url'])
        print("Video link is ",video_url)
        download_url = get_download_url(video_url)
        print("Download link is ",download_url)
        
        df.at[index, 'video_url'] = video_url
        df.at[index, 'download_url'] = download_url

        df.to_csv('video_new_report.csv', index=False)
    else :
        print("Video link for ",row['chapter_name']," is already present")

## Function to download the video to the local SYSTEM

In [2]:
import yt_dlp
import os

def download_video(video_url, filename):
    output_folder = "./downloaded_videos"
    print(video_url)

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    output_path = os.path.join(output_folder, filename)
    ydl_opts = {
        'outtmpl': output_path,  
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([video_url])

def get_file_size(file_path):
    if os.path.exists(file_path):
        return os.path.getsize(file_path)
    else:
        return None



## Save the downloaded video with the chapter name to keep a map of it properly

In [3]:
video_folder = '/Users/abhishek_admin/Desktop/data_migration/downloaded_videos' 

if 'status' not in df.columns:
    df['status'] = None 

if 'video_file_size' not in df.columns:
    df['video_file_size'] = None

for index,row in df.iterrows():
    if row['status'] == "done":
        print(f"Video for {row['chapter_id']} already downloaded")
        continue

    filename_temp = str(row["lesson_id"])
    filename = f"{filename_temp}.mp4"
    download_url_temp = row["download_url"]
    video_url = download_url_temp.replace(" ","")
    print(video_url)
    download_video(video_url,filename)
    print(f"Downloading video for : {filename}")
    df.at[index, 'status'] = "done"
    df.to_csv('red-team-training.csv', index=False)
    chapter_id = str(row['lesson_id'])  
    video_file = os.path.join(video_folder, f'{chapter_id}.mp4')
    video_size = get_file_size(video_file)
    df.at[index, 'video_file_size'] = video_size
    df.to_csv(csv_file, index=False)
    print(f"Appended the file size {video_size} for lesson_id : {chapter_id} to the CSV")

    

Video for 12096753 already downloaded
Video for 12096754 already downloaded
Video for 12067600 already downloaded
Video for 12113718 already downloaded
Video for 12092194 already downloaded
Video for 11969289 already downloaded
Video for 11969289 already downloaded
Video for 11969277 already downloaded
Video for 11969274 already downloaded
Video for 11969250 already downloaded
Video for 11969230 already downloaded
Video for 11969221 already downloaded
Video for 11969210 already downloaded
Video for 11969244 already downloaded
Video for 11961450 already downloaded
Video for 12032114 already downloaded
Video for 12067598 already downloaded
Video for 12113718 already downloaded
Video for 12092185 already downloaded
Video for 12073585 already downloaded
Video for 12073582 already downloaded
Video for 12073584 already downloaded
Video for 12073583 already downloaded
Video for 12113718 already downloaded
Video for 12067587 already downloaded
Video for 12113718 already downloaded
Video for 12